This image is the Android system image which seems to hold the android platform along with bin and config files along with applications.  This area being the main suspect area for third party interference.



In [3]:
# We can use file to find out what kind of image we have.

!file tablet/nandc.boot.img

tablet/nandc.boot.img: Android bootimg, kernel (0x40008000), ramdisk (0x41000000), page size: 2048, cmdline (console=ttyS0,115200 rw init=/init loglevel=4)


In [4]:
# We can mount this image file and investigate what we have and hopefully 
# if we edit anything we can restore the image changes included

!xterm -e sudo mount -o loop tablet/nandd.system.img tablet/system/

In [8]:
# And here we see what is in system folder
!ls -lgG tablet/system/

total 1128
drwxr-xr-x  2    4096 Mar 13 16:44 app
drwxr-xr-x  3    4096 May 23  2014 backup
drwxr-xr-x  3    4096 May 14  2014 bin
-rwxr-xr-x  1    2806 May 14  2014 build.prop
-rwxr-xr-x  1 1087959 May 14  2014 CloudsService.apk
drwxr-xr-x 11    4096 Mar 13 16:37 etc
drwxr-xr-x  2    4096 May 14  2014 fonts
drwxr-xr-x  2    4096 May 14  2014 framework
drwxr-xr-x  7    8192 Mar 13 16:44 lib
drwxr-xr-x  2    4096 May 14  2014 lost+found
drwxr-xr-x  3    4096 May 14  2014 media
drwxr-xr-x  2    4096 May 14  2014 preinstall
drwxr-xr-x  3    4096 May 14  2014 tts
drwxr-xr-x  8    4096 May 14  2014 usr
drwxr-xr-x  4    4096 May 14  2014 vendor
drwxr-xr-x  2    4096 May 14  2014 xbin


In [7]:
# Found this file which is called from the init.rc file
!cat tablet/system/bin/preinstall.sh

#!/system/bin/busybox sh

BUSYBOX="/system/bin/busybox"

/system/bin/chown system:system /dev/block/by-name/misc
/system/bin/chmod 0600 /dev/block/by-name/misc

/system/bin/chown media:radio /dev/mux*
/system/bin/chmod 0660 /dev/mux*

mkdir /boot-resource
mount -t vfat /dev/block/by-name/boot-resource  /boot-resource

if [ ! -e /data/system.notfirstrun ]; then	
    echo "do preinstall job"	
  
	/system/bin/sh /system/bin/pm preinstall /system/preinstall
	/system/bin/sh /system/bin/pm preinstall /sdcard/preinstall

	$BUSYBOX touch /data/system.notfirstrun	
	
	 mkdir /databk
   mount -t ext4 /dev/block/by-name/databk /databk
   rm /databk/data_backup.tar
   umount /databk
   rmdir /databk
   echo "preinstall ok"

elif [ -e /boot-resource/data.need.backup ];then
   echo "data backup:tar /databk/data_backup.tar /data"
   mkdir /databk
   mount -t ext4 /dev/block/by-name/databk /databk
   
   rm /databk/data_backup.tar

   $BUSYBOX tar -cf /databk/data_backup.tar /data
   rm /boot-resource/

In [27]:
# Should be some usefull information here
!cat tablet/system/build.prop